# DSO1: Implémentation et Évaluation de la Méthodologie RLT
## Reinforcement Learning Trees sur Données Multivariées

**Authors:** Dhia Romdhane, Yosri Awedi, Baha Saadoui, Nour Rajhi, Bouguerra Taha, Oumaima Nacef  
**Date:** December 2025  
**Course:** Machine Learning Project - DSO1  
**Méthodologie:** CRISP-DM + RLT (Zhu et al., 2015)

---

## 📚 À propos de ce Notebook (DSO1)

Ce notebook implémente la **méthodologie RLT de base** :
1. **Variable Importance (VI)** - Estimation de l'importance globale des variables
2. **Variable Muting** - Élimination des variables faibles
3. **RLT avec Random Forest** - Modèle embarqué de base
4. **Comparaison Baseline vs RLT** - Évaluation des performances

### 🎯 Scope du DSO1
- ✅ **Modèle Naïf (Baseline)** : Régression/Classification simple
- ✅ **RLT-RandomForest** : Implémentation RLT avec RF comme modèle embarqué
- ✅ **Évaluation complète** : Métriques, visualisations, comparaisons

### 🚀 DSO2 (Travail Futur)
Le **DSO2** explorera d'autres modèles embarqués:
- 🔜 XGBoost
- 🔜 LightGBM
- 🔜 Extra Trees
- 🔜 Gradient Boosting
- 🔜 Neural Networks

### 📊 Datasets Supportés
Ce notebook fonctionne sur **9 datasets** + possibilité d'uploader vos propres données.

---
## 📦 Setup & Configuration

In [ ]:
# Core Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
from datetime import datetime
warnings.filterwarnings('ignore')

# ML Libraries (DSO1: Models simples + Random Forest)
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, KFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import accuracy_score, r2_score, classification_report, confusion_matrix
from sklearn.metrics import mean_squared_error, mean_absolute_error, roc_auc_score
from scipy.stats import f_classif, f_regression

# Configuration
RANDOM_STATE = 42
VI_THRESHOLD = 0.01
np.random.seed(RANDOM_STATE)

# Plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 11

print("="*80)
print("DSO1: Implémentation RLT de Base".center(80))
print("="*80)
print("✓ Bibliothèques importées avec succès!")
print(f"📅 Démarrage: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"\n💡 DSO1 Focus: Baseline + RLT-RandomForest")
print(f"🔜 DSO2 travaillera sur: XGBoost, LightGBM, Extra Trees, etc.")

---
## 📂 Sélection du Dataset

**Option 1:** Choisir un dataset pré-chargé (1-9)  
**Option 2:** Uploader votre propre CSV (0)

In [ ]:
# Datasets disponibles
AVAILABLE_DATASETS = {
    '1': {'file': 'BostonHousing.csv', 'target': 'medv', 'type': 'regression'},
    '2': {'file': 'winequality-red.csv', 'target': 'quality', 'type': 'classification'},
    '3': {'file': 'winequality-white.csv', 'target': 'quality', 'type': 'classification'},
    '4': {'file': 'sonar data.csv', 'target': 'Class', 'type': 'classification'},
    '5': {'file': 'parkinsons.data', 'target': 'status', 'type': 'classification'},
    '6': {'file': 'wdbc.data', 'target': None, 'type': 'classification'},
    '7': {'file': 'auto-mpg.data', 'target': 'mpg', 'type': 'regression'},
    '8': {'file': 'data_school.csv', 'target': None, 'type': 'classification'},
    '9': {'file': 'breast-cancer.csv', 'target': 'diagnosis', 'type': 'classification'}
}

print("📊 DATASETS DISPONIBLES:")
print("="*80)
for key, info in AVAILABLE_DATASETS.items():
    print(f"{key}. {info['file']:<30} Type: {info['type']:<15} Target: {info['target'] or 'Auto'}")
print("\n0. Uploader votre propre CSV")
print("="*80)

In [ ]:
def load_dataset(choice='1'):
    """
    Charger un dataset selon le choix.
    
    Returns:
    --------
    df, target_col, problem_type
    """
    if choice == '0':
        print("📤 Mode Upload: Uploadez votre fichier CSV")
        print("⚠️ Après upload, exécutez la cellule suivante pour traiter le fichier")
        return None, None, None
    
    elif choice in AVAILABLE_DATASETS:
        dataset_info = AVAILABLE_DATASETS[choice]
        filepath = dataset_info['file']
        
        try:
            # Charger le fichier
            if filepath.endswith('.data'):
                df = pd.read_csv(filepath, header=None if 'wdbc' in filepath else 0)
            else:
                df = pd.read_csv(filepath)
            
            # Déterminer la colonne cible
            if dataset_info['target']:
                target_col = dataset_info['target']
            elif 'wdbc' in filepath:
                target_col = df.columns[1]
                df = df.iloc[:, 1:]
            else:
                target_col = df.columns[-1]
            
            problem_type = dataset_info['type']
            
            print(f"✓ Dataset chargé: {filepath}")
            print(f"  Forme: {df.shape}")
            print(f"  Cible: {target_col}")
            print(f"  Type: {problem_type}")
            
            return df, target_col, problem_type
            
        except Exception as e:
            print(f"❌ Erreur lors du chargement de {filepath}: {e}")
            return None, None, None
    else:
        print("❌ Choix invalide")
        return None, None, None

# CHANGEZ CE NUMÉRO pour tester différents datasets (1-9)
DATASET_CHOICE = '1'

df, target_col, problem_type = load_dataset(DATASET_CHOICE)

if df is not None:
    print(f"\n📊 Aperçu du Dataset:")
    display(df.head())

---
## 🔍 Analyse Exploratoire

In [ ]:
if df is not None:
    print("📊 INFORMATIONS SUR LE DATASET")
    print("="*80)
    print(f"Forme: {df.shape[0]} échantillons, {df.shape[1]} caractéristiques")
    print(f"Cible: {target_col}")
    print(f"Type de problème: {problem_type}")
    print(f"\nValeurs manquantes: {df.isnull().sum().sum()}")
    print(f"Doublons: {df.duplicated().sum()}")
    
    print("\n📈 Distribution de la cible:")
    if problem_type == 'classification':
        print(df[target_col].value_counts())
    else:
        print(df[target_col].describe())

In [ ]:
# Visualisations
if df is not None:
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Distribution de la cible
    if problem_type == 'classification':
        df[target_col].value_counts().plot(kind='bar', ax=axes[0], color='steelblue', alpha=0.7)
        axes[0].set_title('Distribution des Classes', fontsize=14, fontweight='bold')
        axes[0].set_ylabel('Nombre')
    else:
        axes[0].hist(df[target_col], bins=30, color='steelblue', edgecolor='black', alpha=0.7)
        axes[0].set_title('Distribution de la Cible', fontsize=14, fontweight='bold')
        axes[0].set_ylabel('Fréquence')
    axes[0].grid(alpha=0.3)
    
    # Heatmap de corrélation
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 1:
        corr_matrix = df[numeric_cols].corr()
        if target_col in corr_matrix.columns:
            top_features = corr_matrix[target_col].abs().nlargest(min(10, len(corr_matrix))).index
            sns.heatmap(df[top_features].corr(), annot=True, fmt='.2f', cmap='coolwarm', 
                       center=0, ax=axes[1], square=True)
            axes[1].set_title('Matrice de Corrélation (Top Features)', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

---
## 🛠️ RLT ÉTAPE 1: Prétraitement des Données

In [ ]:
if df is not None:
    print("🔧 PRÉTRAITEMENT DES DONNÉES")
    print("="*80)
    
    # Séparer features et cible
    X = df.drop(target_col, axis=1)
    y = df[target_col]
    
    # Encoder les variables catégorielles
    categorical_features = X.select_dtypes(exclude=[np.number]).columns
    if len(categorical_features) > 0:
        print(f"⚠️ Encodage de {len(categorical_features)} variables catégorielles...")
        for col in categorical_features:
            le = LabelEncoder()
            X[col] = le.fit_transform(X[col].astype(str))
    
    # Encoder la cible si classification
    if problem_type == 'classification':
        if y.dtype == 'object' or not np.issubdtype(y.dtype, np.number):
            le_target = LabelEncoder()
            y = le_target.fit_transform(y)
    
    # Standardiser les features
    scaler = StandardScaler()
    X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
    
    print(f"\n✓ Prétraitement terminé")
    print(f"  Features: {X_scaled.shape[1]}")
    print(f"  Échantillons: {len(y)}")

---
## 🧠 RLT ÉTAPE 2: Calcul de Variable Importance (VI)

**Méthodologie RLT (Zhu et al., 2015):**
1. Random Forest feature importance (40%)
2. Tests statistiques F-statistic/corrélation (60%)

In [ ]:
if df is not None:
    print("🧠 CALCUL DE VARIABLE IMPORTANCE (VI)")
    print("="*80)
    print("\nMéthode: Random Forest (40%) + Tests Statistiques (60%)")
    
    # Méthode 1: Random Forest VI
    if problem_type == 'classification':
        rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=RANDOM_STATE, n_jobs=-1)
        f_scores, _ = f_classif(X_scaled, y)
    else:
        rf = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=RANDOM_STATE, n_jobs=-1)
        f_scores, _ = f_regression(X_scaled, y)
    
    rf.fit(X_scaled, y)
    vi_rf = rf.feature_importances_
    
    # Méthode 2: VI Statistique
    vi_stat = np.abs(f_scores)
    
    # Normaliser
    vi_rf = vi_rf / vi_rf.sum()
    vi_stat = vi_stat / vi_stat.sum()
    
    # Agréger (RLT DSO1: RF 40% + Stat 60%)
    VI_RF_WEIGHT = 0.4
    VI_STAT_WEIGHT = 0.6
    
    vi_aggregate = VI_RF_WEIGHT * vi_rf + VI_STAT_WEIGHT * vi_stat
    
    # DataFrame VI
    vi_df = pd.DataFrame({
        'Feature': X_scaled.columns,
        'VI_RandomForest': vi_rf,
        'VI_Statistical': vi_stat,
        'VI_Aggregate': vi_aggregate
    }).sort_values('VI_Aggregate', ascending=False)
    
    print("\n📊 Top 10 Features par Importance:")
    display(vi_df.head(10))
    
    print("\n✓ Variable Importance calculée (DSO1: RF + Statistical)")

In [ ]:
# Visualisation VI
if df is not None:
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Top 10 features
    top_10 = vi_df.head(10)
    axes[0].barh(range(len(top_10)), top_10['VI_Aggregate'], color='steelblue', alpha=0.8)
    axes[0].set_yticks(range(len(top_10)))
    axes[0].set_yticklabels(top_10['Feature'])
    axes[0].invert_yaxis()
    axes[0].set_xlabel('Variable Importance (Agrégée)', fontsize=12)
    axes[0].set_title('Top 10 Features - RLT Variable Importance', fontsize=14, fontweight='bold')
    axes[0].grid(axis='x', alpha=0.3)
    
    # Comparaison RF vs Statistical
    x = np.arange(len(top_10))
    width = 0.35
    axes[1].barh(x - width/2, top_10['VI_RandomForest'], width, label='Random Forest (40%)', alpha=0.8)
    axes[1].barh(x + width/2, top_10['VI_Statistical'], width, label='Statistical (60%)', alpha=0.8)
    axes[1].set_yticks(x)
    axes[1].set_yticklabels(top_10['Feature'])
    axes[1].invert_yaxis()
    axes[1].set_xlabel('Variable Importance', fontsize=12)
    axes[1].set_title('Comparaison des Méthodes VI', fontsize=14, fontweight='bold')
    axes[1].legend()
    axes[1].grid(axis='x', alpha=0.3)
    
    plt.tight_layout()
    plt.show()

---
## 🔇 RLT ÉTAPE 3: Variable Muting (Élimination de Variables)

In [ ]:
if df is not None:
    print(f"🔇 APPLICATION DU VARIABLE MUTING (seuil = {VI_THRESHOLD})")
    print("="*80)
    
    # Identifier features à garder
    high_vi_features = vi_df[vi_df['VI_Aggregate'] >= VI_THRESHOLD]['Feature'].tolist()
    low_vi_features = vi_df[vi_df['VI_Aggregate'] < VI_THRESHOLD]['Feature'].tolist()
    
    # Garantir au moins 5 features
    if len(high_vi_features) < 5:
        high_vi_features = vi_df.head(5)['Feature'].tolist()
        low_vi_features = vi_df.iloc[5:]['Feature'].tolist()
        print("⚠️ Moins de 5 features au-dessus du seuil, conservation des 5 meilleures")
    
    # Créer dataset muté
    X_muted = X_scaled[high_vi_features]
    
    muted_count = len(low_vi_features)
    muted_pct = (muted_count / X_scaled.shape[1]) * 100
    
    print(f"\n📊 Résultats du Muting:")
    print(f"  • Features Originales: {X_scaled.shape[1]}")
    print(f"  • Features Conservées: {len(high_vi_features)} ({100-muted_pct:.1f}%)")
    print(f"  • Features Mutées: {muted_count} ({muted_pct:.1f}%)")
    
    if muted_count > 0 and muted_count <= 10:
        print(f"\n🔇 Features Mutées (VI faible):")
        for feat in low_vi_features[:10]:
            vi_value = vi_df[vi_df['Feature'] == feat]['VI_Aggregate'].values[0]
            print(f"    • {feat}: VI = {vi_value:.4f}")
    
    print(f"\n✓ Variable Muting terminé - Réduction: {muted_pct:.1f}%")

---
## 🤖 RLT ÉTAPE 4: Entraînement des Modèles

### DSO1 - Modèles de Base:
1. **Baseline (Naïf)** - Régression/Classification simple sur toutes les features
2. **RLT-RandomForest** - Random Forest sur features mutées

### DSO2 (Futur) - Autres Modèles Embarqués:
- XGBoost, LightGBM, Extra Trees, Gradient Boosting, etc.

In [ ]:
if df is not None:
    print("="*80)
    print("DSO1: ENTRAÎNEMENT DES MODÈLES".center(80))
    print("="*80)
    print("\n📌 Modèles DSO1:")
    print("  1. Baseline (Naïf) - Toutes les features")
    print("  2. RLT-RandomForest - Features mutées")
    print("\n🔜 DSO2 explorera: XGBoost, LightGBM, Extra Trees, etc.")
    print("="*80)
    
    # Configuration selon le type de problème
    if problem_type == 'classification':
        baseline_model = LogisticRegression(max_iter=1000, random_state=RANDOM_STATE)
        rlt_model = RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE, n_jobs=-1)
        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
        scoring = 'accuracy'
        metric_name = 'Accuracy'
    else:
        baseline_model = LinearRegression()
        rlt_model = RandomForestRegressor(n_estimators=100, random_state=RANDOM_STATE, n_jobs=-1)
        cv = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
        scoring = 'r2'
        metric_name = 'R²'
    
    # Entraînement Baseline (toutes features)
    print("\n📊 BASELINE (Modèle Naïf) - Toutes les features:")
    print("-" * 60)
    baseline_scores = cross_val_score(baseline_model, X_scaled, y, cv=cv, scoring=scoring, n_jobs=-1)
    baseline_mean = baseline_scores.mean()
    baseline_std = baseline_scores.std()
    print(f"  {metric_name} = {baseline_mean:.4f} (±{baseline_std:.4f})")
    print(f"  Nombre de features: {X_scaled.shape[1]}")
    
    # Entraînement RLT (features mutées)
    print("\n📊 RLT-RANDOMFOREST - Features mutées:")
    print("-" * 60)
    rlt_scores = cross_val_score(rlt_model, X_muted, y, cv=cv, scoring=scoring, n_jobs=-1)
    rlt_mean = rlt_scores.mean()
    rlt_std = rlt_scores.std()
    print(f"  {metric_name} = {rlt_mean:.4f} (±{rlt_std:.4f})")
    print(f"  Nombre de features: {X_muted.shape[1]}")
    
    # Stocker les résultats
    results = {
        'Baseline': {'mean': baseline_mean, 'std': baseline_std, 'n_features': X_scaled.shape[1]},
        'RLT-RandomForest': {'mean': rlt_mean, 'std': rlt_std, 'n_features': X_muted.shape[1]}
    }

---
## 📊 RLT ÉTAPE 5: Évaluation et Comparaison

In [ ]:
if df is not None:
    print("\n" + "="*80)
    print("COMPARAISON FINALE: BASELINE vs RLT".center(80))
    print("="*80)
    
    # Calcul de l'amélioration
    improvement = ((rlt_mean - baseline_mean) / baseline_mean) * 100
    feature_reduction = muted_pct
    
    # Affichage comparatif
    print(f"\n🏆 RÉSULTATS DSO1:")
    print("-" * 80)
    print(f"  Baseline (Naïf):         {metric_name} = {baseline_mean:.4f} (±{baseline_std:.4f})")
    print(f"                           Features: {X_scaled.shape[1]}")
    print()
    print(f"  RLT-RandomForest:        {metric_name} = {rlt_mean:.4f} (±{rlt_std:.4f})")
    print(f"                           Features: {X_muted.shape[1]} (réduction: {feature_reduction:.1f}%)")
    print("-" * 80)
    
    print(f"\n💡 ANALYSE:")
    print(f"  Amélioration RLT:        {improvement:+.2f}%")
    print(f"  Réduction de features:   {feature_reduction:.1f}%")
    
    winner = "RLT-RandomForest" if rlt_mean > baseline_mean else "Baseline"
    winner_icon = "🎯" if winner == "RLT-RandomForest" else "⚠️"
    print(f"\n{winner_icon} GAGNANT: {winner}")
    
    print("\n" + "="*80)
    print("💬 CONCLUSION DSO1:")
    if rlt_mean > baseline_mean:
        print(f"  ✅ RLT améliore les performances de {improvement:.2f}%")
        print(f"  ✅ Avec {feature_reduction:.1f}% de features en moins!")
    else:
        print(f"  ⚠️ Baseline meilleur de {abs(improvement):.2f}%")
        print(f"  💡 RLT peut mieux fonctionner sur datasets haute dimension")
    print("\n🔜 DSO2 testera d'autres modèles embarqués (XGBoost, LightGBM, etc.)")
    print("="*80)

In [ ]:
# Visualisation de la comparaison
if df is not None:
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Graphique 1: Comparaison des performances
    models = ['Baseline\n(Naïf)', 'RLT-\nRandomForest']
    scores = [baseline_mean, rlt_mean]
    colors = ['steelblue', 'orange']
    
    bars = axes[0].bar(models, scores, color=colors, alpha=0.7, edgecolor='black', width=0.6)
    axes[0].set_ylabel(f'{metric_name} Score', fontsize=12, fontweight='bold')
    axes[0].set_title('DSO1: Baseline vs RLT-RandomForest', fontsize=14, fontweight='bold')
    axes[0].set_ylim([min(scores) * 0.95, max(scores) * 1.05])
    axes[0].grid(axis='y', alpha=0.3)
    
    # Ajouter les valeurs sur les barres
    for bar, score in zip(bars, scores):
        height = bar.get_height()
        axes[0].text(bar.get_x() + bar.get_width()/2., height,
                    f'{score:.4f}', ha='center', va='bottom', 
                    fontsize=12, fontweight='bold')
    
    # Graphique 2: Amélioration et réduction de features
    metrics = ['Performance\nImprovement (%)', 'Feature\nReduction (%)']
    values = [improvement, feature_reduction]
    colors2 = ['green' if improvement > 0 else 'red', 'blue']
    
    bars = axes[1].bar(metrics, values, color=colors2, alpha=0.7, edgecolor='black', width=0.6)
    axes[1].axhline(y=0, color='black', linestyle='-', linewidth=1)
    axes[1].set_ylabel('Pourcentage (%)', fontsize=12, fontweight='bold')
    axes[1].set_title('RLT: Amélioration vs Réduction', fontsize=14, fontweight='bold')
    axes[1].grid(axis='y', alpha=0.3)
    
    # Ajouter les valeurs
    for bar, value in zip(bars, values):
        height = bar.get_height()
        axes[1].text(bar.get_x() + bar.get_width()/2., height,
                    f'{value:+.1f}%' if 'Improvement' in bar.get_x() else f'{value:.1f}%',
                    ha='center', va='bottom' if height > 0 else 'top',
                    fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

---
## 📈 Évaluation Détaillée sur Test Set

In [ ]:
if df is not None:
    print("📈 ÉVALUATION SUR ENSEMBLE DE TEST")
    print("="*80)
    
    # Split train/test
    X_train_full, X_test_full, y_train, y_test = train_test_split(
        X_scaled, y, test_size=0.2, random_state=RANDOM_STATE, 
        stratify=y if problem_type == 'classification' else None
    )
    
    X_train_muted = X_train_full[high_vi_features]
    X_test_muted = X_test_full[high_vi_features]
    
    # Entraîner et prédire
    baseline_model.fit(X_train_full, y_train)
    rlt_model.fit(X_train_muted, y_train)
    
    y_pred_baseline = baseline_model.predict(X_test_full)
    y_pred_rlt = rlt_model.predict(X_test_muted)
    
    # Métriques
    if problem_type == 'classification':
        baseline_score = accuracy_score(y_test, y_pred_baseline)
        rlt_score = accuracy_score(y_test, y_pred_rlt)
        
        print(f"\n🎯 Accuracy sur Test Set:")
        print(f"  Baseline:        {baseline_score:.4f}")
        print(f"  RLT-RandomForest: {rlt_score:.4f}")
        print(f"  Différence:      {(rlt_score - baseline_score):.4f}")
        
        print(f"\n📊 Classification Report (RLT):")
        print(classification_report(y_test, y_pred_rlt))
    else:
        baseline_r2 = r2_score(y_test, y_pred_baseline)
        rlt_r2 = r2_score(y_test, y_pred_rlt)
        
        baseline_rmse = np.sqrt(mean_squared_error(y_test, y_pred_baseline))
        rlt_rmse = np.sqrt(mean_squared_error(y_test, y_pred_rlt))
        
        print(f"\n🎯 Métriques sur Test Set:")
        print(f"\n  R² Score:")
        print(f"    Baseline:         {baseline_r2:.4f}")
        print(f"    RLT-RandomForest: {rlt_r2:.4f}")
        print(f"  \n  RMSE:")
        print(f"    Baseline:         {baseline_rmse:.4f}")
        print(f"    RLT-RandomForest: {rlt_rmse:.4f}")
    
    test_improvement = ((rlt_score if problem_type == 'classification' else rlt_r2) - 
                       (baseline_score if problem_type == 'classification' else baseline_r2)) / \
                      (baseline_score if problem_type == 'classification' else baseline_r2) * 100
    
    print(f"\n💡 Amélioration sur Test: {test_improvement:+.2f}%")

---
## 💾 Sauvegarder les Résultats

In [ ]:
if df is not None:
    # Créer un résumé
    summary = pd.DataFrame({
        'Model': ['Baseline (Naïf)', 'RLT-RandomForest'],
        'Score_Mean': [baseline_mean, rlt_mean],
        'Score_Std': [baseline_std, rlt_std],
        'N_Features': [X_scaled.shape[1], X_muted.shape[1]],
        'Dataset': [AVAILABLE_DATASETS[DATASET_CHOICE]['file']] * 2,
        'Problem_Type': [problem_type] * 2
    })
    
    output_file = f"DSO1_Results_{AVAILABLE_DATASETS[DATASET_CHOICE]['file'].replace('.', '_')}.csv"
    summary.to_csv(output_file, index=False)
    
    print(f"💾 Résultats sauvegardés: {output_file}")
    display(summary)

---
## 📝 Conclusions DSO1

### 🎯 Ce que nous avons accompli (DSO1):

1. **✅ Implémentation RLT complète:**
   - Variable Importance (RF + Statistical)
   - Variable Muting
   - Comparaison Baseline vs RLT

2. **✅ Modèles DSO1:**
   - Baseline Naïf (Logistic/Linear Regression)
   - RLT-RandomForest

3. **✅ Évaluation rigoureuse:**
   - Cross-validation 5-fold
   - Test set evaluation
   - Métriques multiples
   - Visualisations

### 🚀 Pour DSO2 (Travail Futur):

Le **DSO2** explorera d'autres modèles embarqués pour RLT:

**Modèles à tester:**
- 🔜 **XGBoost** - Gradient boosting optimisé
- 🔜 **LightGBM** - Gradient boosting rapide
- 🔜 **Extra Trees** - Variation de Random Forest
- 🔜 **Gradient Boosting** - Boosting classique
- 🔜 **CatBoost** - Pour variables catégorielles
- 🔜 **Neural Networks** - Approche deep learning

**Pistes d'amélioration DSO2:**
- Feature Engineering avancé
- Hyperparameter tuning
- Stacking de modèles
- Feature combinations (interactions)

### 📚 Recommandations:

**Quand utiliser RLT:**
- ✅ Datasets avec > 20 features
- ✅ Présence de variables bruitées
- ✅ Besoin d'interprétabilité
- ✅ Contraintes de vitesse (moins de features)

**Quand éviter RLT:**
- ⚠️ Datasets avec < 10 features
- ⚠️ Toutes les features sont importantes
- ⚠️ Échantillons très petits (n < 100)

---

## 📖 Références

1. **Zhu, R., Zeng, D., & Kosorok, M. R. (2015).** "Reinforcement Learning Trees." *Journal of the American Statistical Association*, 110(512), 1770-1784.

2. **Breiman, L. (2001).** "Random Forests." *Machine Learning*, 45(1), 5-32.

3. **CRISP-DM Methodology** - Cross-Industry Standard Process for Data Mining

---

**Authors:** Dhia Romdhane, Yosri Awedi, Baha Saadoui, Nour Rajhi, Bouguerra Taha, Oumaima Nacef  
**Course:** Machine Learning Project - DSO1  
**Date:** December 2025  
**Repository:** https://github.com/yosriawedi/ML-Project-RLT

---

## 🎉 DSO1 Terminé!

Ce notebook a démontré l'**implémentation et l'évaluation de base de la méthodologie RLT**.

**Prochaine étape:** DSO2 explorera d'autres modèles embarqués! 🚀